In [1]:
import pandas as pd
import numpy as np
import math
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.solvers
import warnings
warnings.filterwarnings(action='ignore')

In [20]:
d1=pd.read_excel(r'final.xlsx')
d1

,Unnamed: 0,Unnamed: 1,A,B,C,D,E,F,G,H,...,N,O,P,Q,R,S,T,U,return,backtest
0,ردیف,شرکت ها,نسبت قیمت به درآمد,نسبت ارزش بازار به دفتری,گردش ح د,گردش موجودی کالا,گردش دارایی ها,نسبت جاری,نسب آنی,بدهی بلندمدت به ح ص س,...,بازده حقوق صاحبان سهام,بازده دارایی ها,حاشیه سود خالص,حاشیه سود عملیاتی,سود هر سهم,نرخ رشد درآمدها,نرخ رشد سود خالص,نرخ رشد بالقوه,بازدهی دوره,بازدهی برون نمونه
1,1,نفت پاسارگاد,26.439778,8.046792,5.902974,17.658152,2.330212,1.251652,1.015429,0.037467,...,0.316201,0.117238,0.048088,0.056362,563.506667,0.912288,1.299972,2.498334,7.335278,10.741667
2,2,پالایش نفت بندر عباس,17.824948,7.974155,70.900909,10.373355,4.477019,1.380752,0.434582,0.024549,...,0.473551,0.245304,0.05648,0.06616,1451.503636,1.833193,1.389427,17.949871,5.610278,33.88
3,3,پالایش نفت اصفهان,18.335863,5.921619,29.898719,17.188804,4.068313,1.79444,0.812496,0.071578,...,0.403398,0.272656,0.069386,0.081216,1329.585363,1.729147,1.824521,12.110027,6.386667,42.138333
4,4,توسعه معدنی و صنعتی صبانور,24.863968,9.980248,5.120275,5.559918,0.550221,1.376582,1.008413,0.055249,...,0.367966,0.253303,0.438904,0.423723,803.453318,1.588731,3.48526,2.167695,6.29,25.071667
5,5,چادرملو,18.917689,7.268893,7.183985,4.242193,0.652577,1.671629,0.951868,0.004396,...,0.366471,0.281938,0.423524,0.38135,781.38175,2.226678,2.939611,2.802514,6.748056,22.323333
6,6,گل گهر,15.953259,7.913687,1.855332,9.112748,0.640822,1.199412,1.005403,0.071554,...,0.486707,0.285308,0.442998,0.420096,1093.826353,2.163888,3.299286,0.932875,6.736944,24.306667
7,7,باما,15.123285,8.041716,2.210327,2.772817,0.460154,8.999733,7.191995,0.001212,...,0.520815,0.473413,1.522414,0.534229,3624.497,0.544242,5.033427,0.838119,7.313056,43.475
8,8,معادن منگنز ایران,32.908883,15.696941,5.841068,11.009882,0.786573,2.152468,1.77849,0.092285,...,0.489597,0.340795,0.410565,0.407723,1570.971673,6.718001,-26.857093,2.936896,11.984722,11.591667
9,9,فجر انرژی خلیج فارس,10.960753,5.24293,1.991196,5.764656,0.61799,1.985145,1.574397,0.118929,...,0.478315,0.301886,0.487457,0.54209,1398.639692,0.250167,0.419606,0.815842,4.044722,18.006667


In [21]:
dg=d1.iloc[1:,2:23].to_numpy()
dg

array([[26.439778095399294, 8.046791741318804, 5.9029740900038865, ...,
        0.9122880886926392, 1.299972449635553, 2.498334007735853],
       [17.824948385844806, 7.974155189893241, 70.90090948815633, ...,
        1.8331929430794036, 1.3894272033236532, 17.94987101562999],
       [18.335862804864984, 5.921619208524386, 29.89871934925687, ...,
        1.7291466137406182, 1.8245214518342097, 12.110026650389232],
       ...,
       [15.318690879629825, 6.228153556222921, 3.0617227909853075, ...,
        0.9631887246641974, 3.0008582047046346, 1.4017793847136197],
       [52.131951888674585, 9.159920174268992, 15.796168987264656, ...,
        1.032538143569029, 2.526354619231084, 7.486918754509197],
       [14.666481825906182, 6.083620617711546, 1.6514359748148462, ...,
        0.9786024827021967, 2.507073445828784, 0.8148763907746258]],
      dtype=object)

In [22]:
w_eff=np.zeros(np.shape(dg))
w_eff.shape

(59, 21)

In [36]:
model=ConcreteModel()
answers={}
for kk in d1.index[1:]:
    #sets
    model.i=Set(initialize=['A','B','C','D','E','F','G','H','I','J','K','L','M'],doc='input index')
    model.r=Set(initialize=['N','O','P','Q','R','S','T','U'],doc='output index')
    model.k=Set(initialize=df.index[1:])
    model.v=Var(model.i, within=NonNegativeReals,doc='input variable')
    model.u=Var(model.r, within=NonNegativeReals,doc='out put variable')
    #objective
    model.objective = Objective(expr=sum(model.u[r]*d1[r][kk] for r in model.r )
                                , sense=maximize, doc='Define objective function')
    #constraint
    model.constraint = Constraint(expr=sum(model.v[i]*d1[i][kk] for i in model.i )==1
                                  , doc='Define first constrint function')
    
    def manager_rule(model,k):
          return (sum(model.u[r]*d1[r][k] for r in model.r )-sum(model.v[i]*d1[i][k] for i in model.i )) <= 0
    model.constraint2 = Constraint(model.k, rule=manager_rule, doc='Define second constrint function')
    
    #solver
    opt = SolverFactory("glpk")
    results = opt.solve(model)
    #answer
    answers[f'CCR of #{kk}']=round(model.objective(),4)
    for i,j in zip(model.i,range(0,13)):
        w_eff[kk-1][j]=round(model.v[i](),4)
    for r,s in zip(model.r,range(14,21)):    
        w_eff[kk-1][s]=round(model.u[r](),4)
    model.clear()

    
#answers

In [57]:
K=np.shape(w_eff)[0]
ce=np.zeros((K,K))
d2=pd.read_excel(r'final.xlsx').iloc[1:,2:23]
d3=d2.iloc[0:,0:13]
d4=d2.iloc[0:,13:]

In [71]:
(ce[i][j]=w_eff[i].dot(d3[j] for j in range(0,K)) for i in range(0,K))


SyntaxError: invalid syntax (2416576343.py, line 1)

In [82]:
w_eff=pd.DataFrame(w_eff)
for i in range(0,K):
    for j in range(0,K):
        ce[i][j]=w_eff.iloc[i].dot(d3.iloc[j])

ValueError: matrices are not aligned

In [83]:
w_eff.iloc[0].dot(d3.iloc[0])

ValueError: matrices are not aligned